In [1]:
import pandas as pd
import plotly.express as px
import altair as alt

In [3]:
readings = pd.read_csv('chemistry_readings.csv', parse_dates=['date'])
chemicals = pd.read_csv('added_chemicals.csv', parse_dates=['date'])
pinch = pd.read_csv('pinch_a_penny.csv', parse_dates=['date'])

### Notes
- w/ Total Alkalinity @ 80, the natural steady state pH is 8.2 
    - why is this a desired level when @ 40, things were steady state cool?
    - getting TA back down to 40 would take 1.08 gallons (!) of muriatic acid

In [4]:
readings.tail(3)

,date,time,total_hardness,total_chlorine,free_chlorine,ph_strip,total_alkalinity,stabilizer,ph_instrument,water_temp
141,2023-04-09,11:02,250.0,0.0,0.0,6.8,60.0,0.0,8.37,78.4
142,2023-04-09,12:13,250.0,0.0,0.0,6.2,40.0,0.0,7.62,78.0
143,2023-04-09,13:14,250.0,0.0,0.0,6.2,40.0,0.0,7.25,79.1


pH visualization

In [5]:
ph_cats = ['good', 'fix', 'rick']
ph_cat_colors = ['green', 'yellow', 'red']
ph_cat_breakpoints = [6.4, 6.8, 7.2, 7.8, 8.2, 8.6]

ph_ranges = pd.DataFrame([
    {'low': ph_cat_breakpoints[0], 'high': ph_cat_breakpoints[1], 'category': ph_cats[2]},
    {'low': ph_cat_breakpoints[1], 'high': ph_cat_breakpoints[2], 'category': ph_cats[1]},
    {'low': ph_cat_breakpoints[2], 'high': ph_cat_breakpoints[3], 'category': ph_cats[0]},
    {'low': ph_cat_breakpoints[3], 'high': ph_cat_breakpoints[4], 'category': ph_cats[1]},
    {'low': ph_cat_breakpoints[4], 'high': ph_cat_breakpoints[5], 'category': ph_cats[2]},
])

In [6]:
base = alt.Chart(readings.groupby('date', as_index=False).mean()).encode(
    x=alt.X('date:T', axis=alt.Axis(format='%m/%d'), title=None),
).properties(
    width=500,
    height=300
)

line = base.mark_line(color='#333').encode(
    alt.Y('ph_instrument:Q', title='pH')
)

points = line.mark_point(color='#333')

zones = alt.Chart(ph_ranges).mark_rect(opacity=.5).encode(
    y=alt.Y('low', scale=alt.Scale(domain=(6.5, 8.5))),
    y2='high',
    color=alt.Color('category:N', scale=alt.Scale(domain=ph_cats, range=ph_cat_colors))
)

# free_chlorine_line = base.mark_line(color='blue').encode(
#     alt.Y('free_chlorine:Q', title='Free Chlorine')
# )

zones + line + points

alt.LayerChart(...)

Chlorine visualization

In [ ]:
fig = px.bar(
    readings.groupby('date', as_index=False).mean(),
    x = 'date',
    y = ['total_chlorine', 'free_chlorine'],
    barmode = 'group'
)

fig.show()

Calculate gallons added from fill time.

In [ ]:
# enter how long the hose was on
duration_of_fill_min = 37
duration_of_fill_sec = 0

# caluclate amount of water added
fill_rate = 5/27  # emperically derived from sophisticated experimenation...it took 27s to fill a 5-gal. bucket
gallons_added = (duration_of_fill_min * 60 + duration_of_fill_sec) * fill_rate
print(f'You added {gallons_added:,.0f} gallons of water.')

In [7]:
chemicals.tail()

,date,time,chemical,amount,unit,note
114,2023-04-04,8:40,water,389.0,gal,35m 0s
115,2023-04-09,11:05,chlorine,1.0,"3"" puck",NaN
116,2023-04-09,11:10,muriatic acid,24.0,oz.,NaN
117,2023-04-09,12:23,muriatic acid,16.0,oz.,NaN
118,2023-04-09,12:54,water,333.0,gal,30m 0s
